# Introduction au reinforcement learning

## AI, ML, ....

L'intelligence artificielle (AI) se caractérise par toutes sortes de méthodes tentant de rendre une machine intelligente. Mais que veut dire rendre une machine intelligente ? Voilà une définition un peu floue...

Un sous-domaine de l'AI est le Machine Learning (apprentissage automatique), qui vise à donner la capacité aux machines d'apprendre à partir de données, leur permettant d'accomplir toutes sortes de tâches.

<img src="img/ai.png" width="780" height="250" align="center"/>

Le ML comporte trois types de learning :
* supervised learning : le programme doit dans ce cas-là apprendre une fonction qui, à partir d'une entrée, donne une sortie. Le programme reçoit pour cela un set d'entraînement comportant des couples (entrée; sortie). Une fois cette fonction apprise, il est alors possible de faire de l'inférence, c'est-à-dire prédire la sortie d'une entrée donnée. Ce champ est le plus répandu des trois dû à différents exploits récents.

<img src="img/supervised.png" width="780" height="50" align="center"/>

<center><em><span style="color:gray">La tâche de classification d'images est un classique du supervised learning</span></em></center>

* unsupervised learning : le but du programme est ici de déterminer une structure dans des données. L'absence de signal de supervisement pour guider le learner distingue cette pratique des méthodes du supervised learning.

<img src="img/unsupervised.png" width="780" height="50" align="center"/>

<center><em><span style="color:gray">Les Generative Adversarial Networks (GAN) sont capables de générer des images de personnes. Cette tâche fait partie du unsupervised learning : l'algorithme n'a eu besoin d'aucun signal de supervisement durant son entraînement.</span></em></center>

* reinforcement learning : le learner est ici un agent, qui interagit avec un environnement en effectuant des actions. Les actions prises par l'agent changent le state de l'environnement et lui procurent un reward. Le learner ne reçoit pas un signal de supervisement lui disant quelles actions choisir à chaque instant : il n'apprend qu'à partir des rewards reçus, qu'il doit maximiser. Ces rewards, une fois maximisées, doivent permettre à l'agent de réussir la tâche souhaitée.

<img src="img/cartpole.gif" width="520" height="50" align="center"/>

<center><em><span style="color:gray">Un agent a ici appris à faire tenir en équilibre un bâton attaché sur un chariot. Il peut choisir entre déplacer le chariot à gauche, à droite, ou ne rien faire. L'agent observer à chaque timestep un state qui décrit la position et vélocité du bâton et du chariot. Enfin, le reward qu'il reçoit est proportionnel à l'angle du bâton : plus le bâton est incliné, plus le reward sera petit.</span></em></center>

Le reinforcement learning peut se diviser en deux parties :
* model-based : l'agent, grâce à son expérience, établit un model de l'environnement grâce à des méthodes de <em>model learning</em>. Il peut ensuite utiliser des méthodes de <em>planning</em> qui, grâce au model appris, permettent de déterminer le comportement optimal.

* model-free : l'agent ne connaît aucune dynamique ni reward de l'environnement. La tâche de learning ne se fait donc qu'à partir de son interaction directe avec l'environnement, l'agent apprend donc via du "trial-and-error". C'est une méthode bien plus applicable que le model based car elle peut fonctionner avec des problèmes où le nombre de states et d'actions sont infinis.

Pour avoir une meilleure compréhension des algorithmes du model-free RL, il est toutefois important de maîtriser les algorithmes model based. C'est pourquoi ils seront d'abord étudiés ici.

Cependant, il ne s'agira ici que d'étudier des méthodes de <planning>. Ainsi, nous partirons de l'assomption que l'agent sait tout sur l'environnement. Il en connaît les dynamiques et peut ainsi prédire les states les plus probables. Il connaît également les rewards, et sait qu'en accomplissant telle action dans tel state lui rapportera tant de reward. La tâche de learning ici ne se fait donc qu'à partir de cette connaissance très précise de l'environnement. Deux problèmes majeurs de cette pratique : la connaissance complète de l'environnement peut être difficile à acquérir dans certains cas, comme lorsque le nombre de states possibles doit être fini, ce qui est très souvent le cas. Le second problème est comme dit précédemment, le fait que le nombre de states et le nombre d'actions possibles doit être fini.

# Markov Decision Process - MDP

## Interaction agent-environnement

Un Markov Decision Process est un framework très utilisé en reinforcement learning car il décrit formellement les interactions entre un agent et un environnement, ainsi que le problème de prise de décision séquentielle. Ainsi, si quelqu'un souhaite résoudre un problème de prise de décision séquentielle, il peut transofrmer ce problème en MDP, et ainsi bénéficier d'outils mais aussi de preuves que la solution optimale sera trouvée. 

* L'<em>agent</em> est abstrait et se caractérise seulement par son algorithme d'apprentissage et de prise de décision.

* L'<em>environnement</em> représente tout ce qui est en dehors de l'agent.

Ces deux entités intéragissent de la façon suivant :

<img src="img/agentenv.png" width="520" height="50" align="center"/>

A chaque timestep $t$ :
* l'agent reçoit le state de l'environnement $S_t$ et le reward $R_t$
* l'agent effectue en conséquence une action $A_t$ sur l'environnement.

Et à $t+1$, l'agent recevra le nouveau state $S_{t+1}$ ainsi qu'un reward $R_{t+1}$, qui dépendent de l'action $A_t$.

Ainsi l'interaction agent-environnement génère une trajectoire :

\begin{equation*}
\large S_0, A_0, R_1, S_1, A_1, R_2, ...
\end{equation*}

Mais cette interaction doit-elle se terminer ou peut elle continuer à l'infini ?

Les deux cas sont possibles : une interaction qui a pour certitude de se terminer est qualifiée d'épisodique.

___

## $M = \; <S, A, P, R>$

Un MDP se définit à partir d'un tuple $M = \; <S, A, P, R>$ :
* $S$ est l'ensemble des states dans lequel peut se trouver l'environnement.
* $A$ est l'ensemble des actions que l'agent peut choisir.
* $P$ représente les dynamiques de l'environnement, c'est-à-dire les probabilités de transition entre chaque state.
* $R$ décrit les rewards obtenus en fonction d'un state et d'une action prise.

$S$ et $A$ sont pour l'instant considérés comme finis.
Les dynamiques de l'environnement $P$ ainsi que les rewards $R$ sont inconnus dans la partie model-free du reinforcement learning.

___

## Dynamiques et propriété markovienne

Les dynamiques de l'environnement se caractérisent par une distribution de probabilité du state successeur $s'$ étant donné le state actuel $s$ et l'action prise $a$ : 

\begin{equation*}
\large p(s'|s,a) = P[S_{t+1} = s' | S_t = s, A_t = a]
\\
\large \sum_{s'} p(s'|s,a) = 1
\end{equation*}

Il est important de constater que le state $s'$ <strong>ne dépend que du state $s$ et de l'action $a$</strong>, et non pas des anciens states. Cette assomption s'appelle la <em>propriété markovienne (Markov property)</em> et est nécessaire pour qu'un MDP soit applicable à un problème :

\begin{equation*}
\large P[S_{t+1} | S_t, A_t, S_{t-1}, A_{t-1} ..., S_0, A_0] = P[S_{t+1} | S_t, A_t]
\end{equation*}

<center><em><span style="color:gray">Autrement dit, le futur est indépendent du passé, compte tenu du présent.</span></em></center>

Beaucoup de problèmes respectent cette propriété. Et même si à première vu un problème ne semble pas le respecter, modifier l'espace des states (pour incorporer l'information manquante) permet de respecter cette propriété, même si cette modification entraînera son aggrandissement.

L'agent connaît donc $p(s'|s,a), \forall s',s, a$, aussi écrit $P_{ss'}^a$.

<em><span style="color:gray">P peut être vu comme un tensor de dimensions $(|A| * |S| * |S|)$, soit $|A|$ matrices de $(|S| * |S|)$ décrivant les probabilités de transitions entre chaque state pour chaque action.</span></em>

___

## Rewards

Les rewards de l'environnement sont définis pour chaque transition, et dépendent du state actuel $s$, de l'action prise $a$, et du state successeur $s'$ :

\begin{equation*}
\large R_{ss'}^a = E[R_{t+1} | S_t=s, A_t=a, S_{t+1}=s']
\end{equation*}

Cette défition générale du reward permet une grande fléxibilité car le reward peut ainsi dépendre de plus de facteurs. Cependant, pour rendre la suite plus intuitive, il est tout aussi raisonnable de penser que le reward reçu à la timestep $t+1$ ne dépend que du state $S_t$ et de l'action $A_t$ :

\begin{equation*}
\large R_{s}^a = E[R_{t+1} | S_t=s, A_t=a]
\end{equation*}

L'agent connaît donc $R_s^a \in R, \forall s, a$.

<em><span style="color:gray">R peut être vu comme une matrice de dimensions $(|S| * |A|)$, soit |A| vecteurs décrivant les rewards reçus pur chaque state pour chaque action.</span></em>

___

## Résoudre un MDP

### Notion de return

Résoudre un MDP revient à avoir un comportement qui maximise l'espérance du cumulative reward, aussi appelé return $G_t$, qui est la somme de tous les rewards que l'agent reçoit à partir de la timestep $t$ :

\begin{equation*}
\large G_t = R_{t+1} + R_{t+2} + R_{t+3} + ... + R_T
\end{equation*}

Comme on peut le remarquer, cette définition de $G_t$ part du principe que l'interaction agent-environnement se termine à la timestep $T$, ce qui n'est pas toujours le cas. Cette défition pourrait être modifiée pour convenir à une interaction infinie, mais alors la variable aléatoire $G_t$ serait inifinie. C'est pourquoi la définition la plus courante de $G_t$ est la suivante :

\begin{align}
\large G_t \; & \large = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ...
\\
& \large = \sum_{k=0}^\infty \gamma ^ k R_{t+k+1}
\end{align}

$\gamma \in [0;1]$ et est un hyperparamètre appelé facteur de réduction (discount factor), qui détermine l'importance des rewards les plus éloignés dans le temps. Ainsi, pour $\gamma=0$, le return $G_t$ ne sera composé que de l'immediate reward $R_{t+1}$, ce qui fait que l'agent agira pour maximiser l'immediate reward seulement et oubliera les rewards futurs. L'agent est alors qualifiée de myope. Au contraire, si $\gamma=1$, l'agent sera hypermétrope car les rewards futurs compteront autant que le reward immédiat.

En plus d'assurer que le return $G_t$ ne soit pas infini, l'introduction du discounting s'inspire du comportement humain et animal qui privilégie les récompenses immédiates.

<center><em>"Mieux vaut un que deux tu l'auras"</em></center>

Il est également important de noter pour la suite que le return $G_t$ peut être décomposé en deux parties : le reward immédiat $R_{t+1}$ et le reste des rewards futurs, $G_{t+1}$, discounted :

\begin{equation*}
\large G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ... = R_{t+1} + \gamma G_{t+1}
\end{equation*}

### Policy

La policy, souvent dénoté $\pi$, d'un agent peut s'apparenter à son comportement : compte tenu d'un state, elle dit à l'agent quelle action choisir.

Toute policy est stochastique, c'est-à-dire que pour tout state $s$, elle associe une probabilité à chaque action $a$: $\pi(a|s)$. Ainsi :

Dans le cas où la policy associe la probabilité de $1$ à une action $a$ et $0$ à toutes les autres pour tout state $s$, la policy est dite déterministique : $a = \pi(s)$

### Value Functions

#### State-value function

Afin de résoudre un MDP, il est utilise de définir la state-value function d'une policy $\pi$, dénotée $v_\pi$. Elle est définie pour tout state $s$ et est égale à l'espérance du return à partir du state $s$, si l'agent suit la policy $\pi$.

\begin{equation*}
\large v_\pi(s) = E_\pi[G_t | S_t = s]
\end{equation*}

<em><span style="color:gray">
Pourquoi la state-value function dépend d'une policy $\pi$ ? 
</br>
$v_\pi$ est une manière d'évaluer $\pi$, de savoir, en moyenne, le return qu'elle procure pour chaque state.
</span></em>

#### Action-value function

De même, on peut définir l'action-value function d'une policy $\pi$, dénotée $q_\pi$. Elle est définie pour tout state $s$ et action $a$, et est égale à l'espérance du return si l'agent exécute $a$ dans $s$ et suit ensuite $\pi$.

\begin{equation*}
\large q_\pi(s,a) = E_\pi[G_t | S_t = s, A_t = a]
\end{equation*}

### Bellman expectation equations

Les équations de Bellman sont un outil très utilisé dans le reinforcement learning, à la fois model-based et model-free.

Ici, les équations d'expectation de Bellman vont servir à approximer les fonctions $v_\pi$ et $q_\pi$.

#### Exprimer récursivement $v_\pi(s)$

Comme vu précédemment, $G_t = R_{t+1} + \gamma G_{t+1}$. Ainsi, on peut écrire :

\begin{equation*}
\large v_\pi(s) = E_\pi[G_t | S_t = s] = E_\pi[R_{t+1} + \gamma G_t | S_t = s] = E_\pi[R_{t+1} | S_t = s] + \gamma E_\pi[G_{t+1} | S_t = s]
\end{equation*}

$R_{t+1}$ correspond au reward immédiat, et $G_{t+1}$ à la value du state successeur de $s$.

Calculer $E_\pi[R_{t+1} | S_t = s]$ ainsi que $E_\pi[G_{t+1} | S_t = s]$ permettra donc de calculer $v_\pi(s)$.

$E_\pi[R_{t+1} | S_t = s]$ représente le reward immédiat moyen que l'agent reçoit lorsqu'il sort du state $s$ et suit $\pi$.

<img src="img/expected_r_t+1.png" width="300" height="50" align="center"/>

Comme montré dans le <em>backup diagram</em> ci-dessus, l'agent peut recevoir un parmis $|A|$ possibles immediate rewards en sortant du state $s$. Chaque reward $R_s^a$ a une probabilité $\pi(a|s)$ d'être reçu par l'agent. Ainsi, l'espérance (ou moyenne) du reward immédiat $R_{t+1}$ du state $s$ se calcule :

\begin{equation*}
\large E_\pi[R_{t+1} | S_t = s] =  \sum_a \pi(a|s) R_s^a
\end{equation*}

$E_\pi[G_{t+1} | S_t = s]$ représente le reward futur moyen que l'agent peut recevoir lorsqu'il est dans le state $s$, et suit $\pi$.

<img src="img/expected_g_t+1.png" width="500" height="50" align="center"/>

Le diagramme ci-dessus montre que, dans le state $s$, l'agent peut avoir différents futur return $G_{t+1}$, chacun associé à une probabilité d'arriver, tout comme l'agent pouvait précédemment recevoir plusieurs rewards immédiats $R_{t+1}$. La probabilité d'avoir un certain return $v_\pi(s')$ est d'abord la probabilité $\pi(a|s)$ de prendre l'action $a$ qui mène l'agent "à portée" de $s'$ multipliée par la probabilité $p(s'|s,a)$ de transitionner, compte tenu de $s$ et $a$, vers le state $s'$. Ainsi, la somme de ces probabilités avec leur valeur correspondante donne la définition suivante de $E_\pi[G_{t+1}]$:

\begin{equation*}
\large E_\pi[G_{t+1} | S_t = s] = \sum_a \pi(a|s) \sum_{s'} p(s'|s,a) v_\pi(s')
\end{equation*}

On peut donc écrire :

\begin{align}
\large v_\pi(s) \; & \large = E_\pi[R_{t+1} | S_t = s] + \gamma E_\pi[G_{t+1} | S_t = s]
\\
& \large = \sum_a \pi(a|s) R_s^a + \gamma \sum_a \pi(a|s) \sum_{s'} p(s'|s,a) v_\pi(s')
\\
& \large = \sum_a \pi(a|s) [R_s^a + \gamma \sum_{s'} p(s'|s,a) v_\pi(s')]
\end{align}

<center><em><span style="color:gray">Equation d'expectation de Bellman pour $v_\pi(s)$</span></em></center>

#### Exprimer récursivement $q_\pi(s, a)$

Comme vu précédemment, $G_t = R_{t+1} + \gamma G_{t+1}$. Ainsi, on peut écrire : 

\begin{equation*}
\large q_\pi(s,a) = E_\pi[G_t | S_t = s, A_t = a] = E_\pi[R_{t+1} + \gamma G_t | S_t = s, A_t = a] = E[R_{t+1} | S_t = s, A_t =a ] + \gamma E_\pi[G_{t+1} | S_t = s, A_t = a]
\end{equation*}

Calculer $E[R_{t+1} | S_t=s, A_t=a]$ ainsi que $E_\pi[G_{t+1} | S_t=s, A_t=a]$ permettra donc de calculer $q_\pi(s, a)$.

$E[R_{t+1} | S_t=s, A_t=a]$ représente l'espérance du reward immédiat étant donné le current state $s$ et l'action prise $a$. Cette valeur est fournie à l'agent par l'environnement : c'est $R_s^a$. Aucun calcul n'est donc nécessaire car l'agent sait qu'il va prendre l'action $a$ avec une probabilité de $1$.

<img src="img/q_expected_r_t+1.png" width="300" height="50" align="center"/>

$E_\pi[G_{t+1} | S_t=s, A_t=a]$ représente le reward futur moyen que l'agent peut recevoir lorsqu'il est dans le state $s$, prend l'action $a$, et suit ensuite $\pi$.

<img src="img/q_expected_g_t+1.png" width="500" height="50" align="center"/>

Le raisonnement pour calculer $E_\pi[G_{t+1} | S_t=s, A_t=a]$ est similaire à celui utilisé avec $v_\pi(s)$ mais est même plus simple car l'agent sait quelle action $a$ il va prendre. Le diagramme montre donc que l'agent peut avoir différents futur return $G_{t+1}$ chacun associé à une probabilité d'arriver. La probabilité d'avoir un certain futur return $v_\pi(s')$ est égale à la probabilité $p(s'|s,a)$ de transitionner, compte tenu de $s$ et $a$, vers le state $s'$. Ainsi : 

\begin{equation*}
\large E_\pi[G_{t+1} | S_t = s, A_t = a] = 1 \sum_{s'} p(s'|s,a) v_\pi(s')
\end{equation*}

Ainsi :

\begin{align}
\large q_\pi(s,a) & \large = E[R_{t+1} | S_t = s | A_t = a] + \gamma E_\pi[G_{t+1} | S_t = s, A_t = a]
\\
&
\large \; = R_s^a + \gamma \sum_{s'} p(s'|s,a) v_\pi(s')
\end{align}

### Optimalité

#### $\pi_*$ :

La policy optimale $\pi_*$ est par définition la policy qui, pour tout state $s$ et pour toute policy $\pi$ :

\begin{equation*}
\large v_{\pi_*}(s) \geq v_\pi(s), \quad \forall \pi,s
\end{equation*}

Il est très souvent possible qu'il existe une multitude de policy optimale, voir même une infinité de policy optimales stochastiques.

#### $v_*$ :

La state-value function optimale $v_*$ à la propriété suivante :

\begin{equation*}
\large v_*(s) = v_{\pi_*}(s) = max_\pi v_\pi(s), \quad \forall s
\end{equation*}

Il n'existe qu'une seule statue-value optimale.

#### $q_*$ :

La action-value function optimale $q_*$ à la propriété suivante :

\begin{equation*}
\large q_*(s,a) = q_{\pi_*}(s,a) = max_\pi q_\pi(s,a), \quad \forall s,a
\end{equation*}

<center><em><span style="color:red">Attention : $a$ n'est pas forcément l'action optimale à prendre dans le state $s$</span></em></center>

#### Exprimer récursivement $v_*$ et $q_(s,a)$

<img src="img/backup_v_star.png" width="500" height="50" align="center"/>

Le trait rouge présent sur le diagramme signifie que $v_*(s)$ prendra la valeur la plus grande entre les deux branches. Cela se traduit sous forme d'équation ainsi :

\begin{equation*}
\large v_*(s) = max_a [R_s^a + \gamma \sum_{s'} p(s'|s,a) v_*(s')]
\end{equation*}

Ainsi, si $v_*$ est calculée, la valeur de tout state $s$ sera la maximale car $v_*(s)$ utilise l'action optimale du state $s$.

___

<img src="img/backup_q_star.png" width="1000" height="50" align="center"/>

Le principe s'applique de la même manière avec $q_*(s,a)$ :

\begin{align}
\large q_*(s,a) & \large = R_s^a + \gamma \sum_{s'} p(s'|s,a) v_*(s')
\\
&
\large \; = R_s^a + \gamma \sum_{s'} p(s'|s,a) max_{a'} q_*(s',a')
\end{align}

Le max est ici pris lorsque l'agent prend l'action $a'$ qui est l'action optimale dans le state $s'$.

### Calculer la state-value function

Le MDP est considéré comme résolu lorsque la state-value function $v_*$ est trouvée. Il existe une méthode appelée programmation dynamique, introduite par Bellman, qui est très utilisée pour résoudre un MDP. Généralement, le dynamic programming vise à résoudre un problème et pour cela, le décompose en plusieurs sub-problems, qu'il résout chacun d'un côté. La même idée s'applique au MDP : trouver un comportement optimal revient à d'abord trouver la meilleure action optimale à prendre maintenant puis toutes les suivantes.

L'algorithme de <strong>Policy Evaluation</strong> vise à calculer $v_\pi$ et s'appuie pour cela sur l'équation d'expectation de Bellman.
</br>L'algorithme de <strong> Policy Improvement</strong> vise à améliorer une policy à partir de sa value function, il s'appuie sur l'équation d'optimality de Bellman.
</br>Enfin l'algorithme de <strong>Value Iteration</strong> est une application itérative de l'équation d'optimality de Bellman et calcule directement $v_*$.

___

# Conclusion

Le Reinforcement Learning implique donc l'interaction d'un agent (le learner) et d'un environnement. L'agent effectue des actions, voit le state de l'environnement changer et reçoit des rewards, qu'il tente de maximiser. L'agent peut tout connaître de son environnement, ainsi la tâche est appelée model-based learning, et model-free learning dans le cas contraire. Nous nous intéressons pour commencer au cas du model-based learning, en utilisant des tableaux pour les fonctions $\pi, v, q$.
</br>
Un MDP est une définition formelle de l'interaction agent-environnement, qui permet d'assurer la convergence sur la solution optimale. Il est définit, dans le cas du model-based learning, à partir d'un state space qui possèdent la propriété markovienne, un action space, des dynamiques et des rewards. Le comportement de l'agent est définit par sa policy $\pi$, qui est évaluée grâce à une state-value function $v_\pi$ ou encore grâce à une action-value function $q_\pi$.
</br>
Les équations de Bellman proposent une définition récursive de $v_\pi$, $q_\pi$, $v_*$ et $q_*$, qui servent de base pour calculer ces fonctions et ainsi résoudre le MDP. Il s'agit seulement d'un développement d'une espérance, donc on fait la somme de toutes les valuers possibles, pondérées par leur probabilité d'arriver.

